# Chapter 21: Events in JavaScript

---

## Introduction

**Events** are the mechanism that enables JavaScript to interact with HTML documents and respond to user actions. Every interaction on a web page—whether it's a mouse click, a key press, a form submission, or the completion of a page load—triggers an event that your JavaScript code can listen for and respond to.

The event-driven programming model is fundamental to frontend development. Unlike traditional linear programs that execute from top to bottom, event-driven applications wait for events to occur and then execute specific code in response. This model is essential for creating interactive, responsive user interfaces that feel natural and immediate to users.

Understanding events thoroughly requires knowledge of how to register event listeners, how events propagate through the DOM tree, how to access event information through the event object, and how to control event behavior. Mastering these concepts allows you to build sophisticated interactions while maintaining clean, efficient, and accessible code.

In this chapter, you will learn the complete event handling system in JavaScript, from basic event registration to advanced patterns like event delegation and custom events.

---

## 21.1 Introduction to Events

### What Are Events?

An **event** is a signal that something has happened in the browser. Events can be triggered by:

- **User interactions**: Clicking a button, typing in a field, moving the mouse, resizing the window
- **Browser actions**: Page finishing load, image completing download, error occurring
- **API triggers**: Animation completing, data arriving from server, timer expiring
- **Scripted events**: Programmatically triggering events using JavaScript

### Event Types Overview

```javascript
// Mouse Events
click       // Single click
dblclick    // Double click
mousedown   // Mouse button pressed
mouseup     // Mouse button released
mousemove   // Mouse moved
mouseenter  // Mouse enters element (no bubbling)
mouseleave  // Mouse leaves element (no bubbling)
mouseover   // Mouse enters element (bubbles)
mouseout    // Mouse leaves element (bubbles)
contextmenu // Right-click menu

// Keyboard Events
keydown     // Key pressed down
keypress    // Key pressed (deprecated, use keydown)
keyup       // Key released

// Form Events
focus       // Element receives focus
blur        // Element loses focus
change      // Value changed and element loses focus
input       // Value changed (immediate)
submit      // Form submitted
reset       // Form reset
select      // Text selected

// Window/Document Events
load        // Resource loaded
DOMContentLoaded // DOM fully parsed
resize      // Window resized
scroll      // Element scrolled
error       // Error occurred
beforeunload // Before leaving page
unload      // Page unloading

// Touch Events (mobile)
touchstart  // Touch begins
touchmove   // Touch moves
touchend    // Touch ends
touchcancel // Touch cancelled

// Drag and Drop
dragstart
drag
dragenter
dragleave
dragover
drop
dragend
```

### Event-Driven Architecture

```javascript
// Traditional procedural approach (not event-driven)
function runProgram() {
    console.log("Step 1");
    console.log("Step 2");
    console.log("Step 3");
}
runProgram(); // Executes immediately in order

// Event-driven approach
function setupEventListeners() {
    // Program waits for events, then responds
    document.getElementById('startBtn').addEventListener('click', function() {
        console.log("Start button clicked!");
    });
    
    document.getElementById('inputField').addEventListener('input', function(event) {
        console.log("Input changed:", event.target.value);
    });
    
    window.addEventListener('resize', function() {
        console.log("Window resized to:", window.innerWidth, "x", window.innerHeight);
    });
}

// Setup listeners, then wait for user interaction
setupEventListeners();
console.log("Event listeners ready. Waiting for user interaction...");
```

---

## 21.2 Adding Event Listeners

The modern way to handle events in JavaScript is using `addEventListener()`. This method attaches an event handler to an element without overwriting existing handlers, unlike the older `onclick` property approach.

### addEventListener Syntax

```javascript
// Basic syntax
element.addEventListener(event, listener, options);

// Parameters:
// event: String - event type (e.g., 'click', 'submit')
// listener: Function - callback function to execute
// options: Object or Boolean (optional) - capture, once, passive

// Basic example
const button = document.getElementById('myButton');

button.addEventListener('click', function() {
    console.log('Button was clicked!');
});

// With named function (better for removal)
function handleClick() {
    console.log('Handled click');
}

button.addEventListener('click', handleClick);
```

### Multiple Event Listeners

```javascript
const button = document.getElementById('actionBtn');

// Multiple listeners on same element, same event
button.addEventListener('click', function() {
    console.log('First handler');
});

button.addEventListener('click', function() {
    console.log('Second handler');
});

button.addEventListener('click', function() {
    console.log('Third handler');
});

// All three execute in order when button is clicked
// Output: First handler, Second handler, Third handler

// Different events on same element
const input = document.getElementById('username');

input.addEventListener('focus', function() {
    console.log('Input focused');
});

input.addEventListener('blur', function() {
    console.log('Input lost focus');
});

input.addEventListener('input', function(event) {
    console.log('Value changed to:', event.target.value);
});
```

### Removing Event Listeners

```javascript
const button = document.getElementById('toggleBtn');

function handleClick() {
    console.log('Button clicked');
}

// Add listener
button.addEventListener('click', handleClick);

// Remove listener (must use same function reference)
button.removeEventListener('click', handleClick);

// Anonymous functions cannot be removed
button.addEventListener('click', function() {
    console.log('Anonymous');
});
// Cannot remove the above listener!

// One-time listener using { once: true }
button.addEventListener('click', function() {
    console.log('This only runs once');
}, { once: true });

// Or manually remove after first execution
function runOnce() {
    console.log('Running once');
    button.removeEventListener('click', runOnce);
}
button.addEventListener('click', runOnce);
```

### Event Listener Options

```javascript
const element = document.getElementById('target');

// Option 1: capture (boolean)
// Event flows from document down to target (capture phase) 
// then back up (bubble phase). Default is bubble.
element.addEventListener('click', handler, true); // Capture phase
element.addEventListener('click', handler, false); // Bubble phase (default)

// Option 2: once (boolean)
// Listener automatically removed after first invocation
element.addEventListener('click', function() {
    console.log('This runs only once');
}, { once: true });

// Option 3: passive (boolean)
// Indicates listener will never call preventDefault()
// Improves scroll performance on touch/wheel events
element.addEventListener('touchstart', handleTouch, { passive: true });
element.addEventListener('wheel', handleWheel, { passive: true });

// Option 4: signal (AbortSignal)
// Allows removing multiple listeners at once
const controller = new AbortController();
const { signal } = controller;

element.addEventListener('click', handler1, { signal });
element.addEventListener('mouseover', handler2, { signal });
element.addEventListener('focus', handler3, { signal });

// Remove all three listeners at once
controller.abort();
```

### The Event Object

When an event occurs, the browser creates an **event object** containing information about the event and passes it to the event handler:

```javascript
const button = document.getElementById('actionBtn');

button.addEventListener('click', function(event) {
    // event (or e) is the event object
    console.log(event); // MouseEvent { ... }
    
    // Common event properties
    console.log(event.type);      // "click"
    console.log(event.target);    // Element that was clicked
    console.log(event.currentTarget); // Element that has the listener
    console.log(event.timeStamp); // Time when event occurred
    
    // Mouse-specific properties
    console.log(event.clientX);   // X coordinate relative to viewport
    console.log(event.clientY);   // Y coordinate relative to viewport
    console.log(event.pageX);     // X coordinate relative to document
    console.log(event.pageY);     // Y coordinate relative to document
    console.log(event.button);    // Which mouse button (0=left, 1=middle, 2=right)
});
```

### Event Object Methods

```javascript
element.addEventListener('click', function(event) {
    // Prevent default browser behavior
    event.preventDefault();
    // Example: Prevent form submission or link navigation
    
    // Stop event propagation (bubbling)
    event.stopPropagation();
    // Prevents event from reaching parent elements
    
    // Stop immediate propagation
    event.stopImmediatePropagation();
    // Also prevents other listeners on same element from firing
    
    // Check if preventDefault was called
    if (event.defaultPrevented) {
        console.log('Default was prevented');
    }
});
```

---

## Chapter Summary

In this chapter, you learned the comprehensive event handling system in JavaScript:

1. **Event Fundamentals**: Events are signals that something has happened in the browser, triggered by user interactions, browser actions, or scripts. The event-driven model is fundamental to interactive web applications.

2. **Adding Event Listeners**: The `addEventListener()` method is the modern standard for attaching event handlers. It allows multiple listeners on the same element, provides options for capture phase, once-only execution, and passive listeners, and can be removed with `removeEventListener()`.

3. **The Event Object**: When events fire, the browser passes an event object containing detailed information about the event—type, target, coordinates, key information, and methods to control event behavior like `preventDefault()` and `stopPropagation()`.

4. **Event Types**: JavaScript supports numerous event categories including mouse events (click, mousemove, mouseenter), keyboard events (keydown, keyup), form events (submit, input, change), and window events (load, resize, scroll).

5. **Event Propagation**: Events flow through the DOM in two phases—capturing (down from document to target) and bubbling (up from target to document). Understanding propagation is essential for event delegation and preventing unwanted side effects.

6. **Event Delegation**: A performance optimization pattern where a single event listener on a parent element handles events for all its children, using `event.target` to determine which child was interacted with. Essential for dynamic content.

7. **Preventing Default Behavior**: The `preventDefault()` method stops the browser's default action (like form submission or link navigation), allowing JavaScript to handle the action instead.

8. **Removing Event Listeners**: Proper cleanup of event listeners is crucial for preventing memory leaks, especially in single-page applications. Listeners must be removed using the same function reference used for attachment.

### Key Takeaways

- Always use `addEventListener` instead of inline event handlers (`onclick` attributes)
- Use event delegation for handling events on multiple similar elements or dynamic content
- Understand the difference between `event.target` (element that triggered event) and `event.currentTarget` (element with the listener)
- Call `preventDefault()` when you want to handle an action in JavaScript instead of letting the browser handle it
- Remove event listeners when elements are removed from the DOM to prevent memory leaks
- Use the `{ once: true }` option for listeners that should only fire once

### Practice Exercises

1. Create a todo list application using event delegation where clicking a todo item toggles its completed status and clicking a delete button removes the item.

2. Implement a form validation system that validates fields in real-time as the user types, showing error messages immediately below invalid fields.

3. Build an image gallery with keyboard navigation (arrow keys to navigate, Escape to close) and ensure all interactions are accessible.

4. Create a drag-and-drop file upload zone that handles dragenter, dragover, dragleave, and drop events with visual feedback during dragging.

5. Implement an infinite scroll mechanism that loads more content when the user scrolls near the bottom of the page, using scroll and resize events efficiently with throttling.

---

## Coming Up Next

**Chapter 22: Forms and JavaScript**

In the next chapter, you will learn how to work with HTML forms using JavaScript, covering:

- Accessing form elements and their values through the DOM
- Handling form submission events and preventing default submission
- Real-time form validation using input and change events
- Working with different input types (text, checkbox, radio, select, file)
- The Constraint Validation API for native form validation
- Creating custom validation logic and error messaging
- Serializing form data for submission to servers
- Managing form state and dynamic form fields

Forms are the primary mechanism for user input on the web, making this chapter crucial for building interactive applications that collect and process user data.

---